In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
%matplotlib inline


Transfer Learning for Computer Vision Tutorial
==============================================

These two major transfer learning scenarios look as follows:

-  **Finetuning the convnet**: Instead of random initialization, we
   initialize the network with a pretrained network, like the one that is
   trained on imagenet 1000 dataset. Rest of the training looks as
   usual.
-  **ConvNet as fixed feature extractor**: Here, we will freeze the weights
   for all of the network except that of the final fully connected
   layer. This last fully connected layer is replaced with a new one
   with random weights and only this layer is trained.


In [3]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode

Load Data
---------

We will use torchvision and torch.utils.data packages for loading the
data.

The problem we're going to solve today is to train a model to classify
**ants** and **bees**. We have about 120 training images each for ants and bees.
There are 75 validation images for each class. Usually, this is a very
small dataset to generalize upon, if trained from scratch. Since we
are using transfer learning, we should be able to generalize reasonably
well.

This dataset is a very small subset of imagenet.

.. Note ::
   Download the data from
   `here <https://download.pytorch.org/tutorial/hymenoptera_data.zip>`_
   and extract it to the current directory.



In [4]:
# Data augmentation and normalization for training
# Just normalization for validation
image_transforms  = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.Resize(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),
        transforms.Resize(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

dataset = 'microsphere2'
train_directory = os.path.join(dataset, 'train')
valid_directory = os.path.join(dataset, 'valid')

batch_size = 32
num_classes = 173

data = {
    'train': datasets.ImageFolder(root=train_directory, transform=image_transforms['train']),
    'valid': datasets.ImageFolder(root=valid_directory, transform=image_transforms['valid'])

}


train_data_size = len(data['train'])
valid_data_size = len(data['valid'])

train_data = DataLoader(data['train'], batch_size=batch_size, shuffle=True)
valid_data = DataLoader(data['valid'], batch_size=batch_size, shuffle=True)

print(train_data_size, valid_data_size)

1038 346


迁移学习
---------


这里使用ResNet-50的预训练模型。


In [5]:
resnet50 = models.resnet50(pretrained=True)

for param in resnet50.parameters():
    param.requires_grad = True

fc_inputs = resnet50.fc.in_features
resnet50.fc = nn.Sequential(
    nn.Linear(fc_inputs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(64, 240),
    nn.LogSoftmax(dim = 1)


)
resnet50 = resnet50.to('cuda:0')
loss_func = nn.NLLLoss()
optimizer = optim.AdamW(resnet50.parameters(),lr=1e-4)

Training the model
------------------

Now, let's write a general function to train a model. Here, we will
illustrate:

-  Scheduling the learning rate
-  Saving the best model

In the following, parameter ``scheduler`` is an LR scheduler object from
``torch.optim.lr_scheduler``.



In [6]:
def train_and_valid(model, loss_function, optimizer, epochs=25):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    history = []
    best_acc = 0.0
    best_epoch = 0

    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))

        model.train()

        train_loss = 0.0
        train_acc = 0.0
        valid_loss = 0.0
        valid_acc = 0.0

        for i, (inputs, labels) in enumerate(train_data):
            inputs = inputs.to(device)
            labels = labels.to(device)

            #因为这里梯度是累加的，所以每次记得清零
            optimizer.zero_grad()

            outputs = model(inputs)

            loss = loss_function(outputs, labels)

            loss.backward()

            optimizer.step()

            train_loss += loss.item() * inputs.size(0)

            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))

            acc = torch.mean(correct_counts.type(torch.FloatTensor))

            train_acc += acc.item() * inputs.size(0)

        with torch.no_grad():
            model.eval()

            for j, (inputs, labels) in enumerate(valid_data):
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)

                loss = loss_function(outputs, labels)

                valid_loss += loss.item() * inputs.size(0)

                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))

                acc = torch.mean(correct_counts.type(torch.FloatTensor))

                valid_acc += acc.item() * inputs.size(0)

        avg_train_loss = train_loss/train_data_size
        avg_train_acc = train_acc/train_data_size

        avg_valid_loss = valid_loss/valid_data_size
        avg_valid_acc = valid_acc/valid_data_size

        history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])

        if best_acc < avg_valid_acc:
            best_acc = avg_valid_acc
            best_epoch = epoch + 1

        epoch_end = time.time()

        print("Epoch: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation: Loss: {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(
            epoch+1, avg_valid_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start
        ))
        print("Best Accuracy for validation : {:.4f} at epoch {:03d}".format(best_acc, best_epoch))

        torch.save(model, 'models/'+dataset+'_model_'+str(epoch+1)+'.pt')
    return model, history


In [7]:
num_epochs = 3000
trained_model, history = train_and_valid(resnet50, loss_func, optimizer, num_epochs)
torch.save(history, 'models/'+dataset+'_history.pt')



Epoch: 1/3000
Epoch: 001, Training: Loss: 5.4774, Accuracy: 0.4817%, 
		Validation: Loss: 5.4774, Accuracy: 0.5780%, Time: 632.0046s
Best Accuracy for validation : 0.0058 at epoch 001
Epoch: 2/3000
Epoch: 002, Training: Loss: 5.4693, Accuracy: 0.3854%, 
		Validation: Loss: 5.4693, Accuracy: 0.5780%, Time: 425.2935s
Best Accuracy for validation : 0.0058 at epoch 001
Epoch: 3/3000
Epoch: 003, Training: Loss: 5.4649, Accuracy: 0.5780%, 
		Validation: Loss: 5.4649, Accuracy: 0.5780%, Time: 26.9697s
Best Accuracy for validation : 0.0058 at epoch 001
Epoch: 4/3000
Epoch: 004, Training: Loss: 5.4338, Accuracy: 0.8671%, 
		Validation: Loss: 5.4338, Accuracy: 0.5780%, Time: 13.9357s
Best Accuracy for validation : 0.0058 at epoch 001
Epoch: 5/3000
Epoch: 005, Training: Loss: 5.3889, Accuracy: 1.0597%, 
		Validation: Loss: 5.3889, Accuracy: 1.7341%, Time: 13.7163s
Best Accuracy for validation : 0.0173 at epoch 005
Epoch: 6/3000
Epoch: 006, Training: Loss: 5.3282, Accuracy: 1.2524%, 
		Validation:

Epoch: 46/3000
Epoch: 046, Training: Loss: 3.8865, Accuracy: 3.6609%, 
		Validation: Loss: 3.8865, Accuracy: 6.3584%, Time: 14.2402s
Best Accuracy for validation : 0.0636 at epoch 046
Epoch: 47/3000
Epoch: 047, Training: Loss: 3.8861, Accuracy: 2.2158%, 
		Validation: Loss: 3.8861, Accuracy: 6.6474%, Time: 15.1718s
Best Accuracy for validation : 0.0665 at epoch 047
Epoch: 48/3000
Epoch: 048, Training: Loss: 3.9162, Accuracy: 2.5048%, 
		Validation: Loss: 3.9162, Accuracy: 6.0694%, Time: 14.2452s
Best Accuracy for validation : 0.0665 at epoch 047
Epoch: 49/3000
Epoch: 049, Training: Loss: 3.8757, Accuracy: 2.7938%, 
		Validation: Loss: 3.8757, Accuracy: 6.0694%, Time: 14.0899s
Best Accuracy for validation : 0.0665 at epoch 047
Epoch: 50/3000
Epoch: 050, Training: Loss: 3.8704, Accuracy: 3.2755%, 
		Validation: Loss: 3.8704, Accuracy: 4.9133%, Time: 13.9438s
Best Accuracy for validation : 0.0665 at epoch 047
Epoch: 51/3000
Epoch: 051, Training: Loss: 4.3152, Accuracy: 3.7572%, 
		Validat

Epoch: 091, Training: Loss: 3.2793, Accuracy: 6.1657%, 
		Validation: Loss: 3.2793, Accuracy: 10.4046%, Time: 13.8505s
Best Accuracy for validation : 0.1272 at epoch 088
Epoch: 92/3000
Epoch: 092, Training: Loss: 3.1800, Accuracy: 5.4913%, 
		Validation: Loss: 3.1800, Accuracy: 10.4046%, Time: 13.7781s
Best Accuracy for validation : 0.1272 at epoch 088
Epoch: 93/3000
Epoch: 093, Training: Loss: 3.3362, Accuracy: 7.1291%, 
		Validation: Loss: 3.3362, Accuracy: 10.4046%, Time: 13.7932s
Best Accuracy for validation : 0.1272 at epoch 088
Epoch: 94/3000
Epoch: 094, Training: Loss: 3.2293, Accuracy: 5.2987%, 
		Validation: Loss: 3.2293, Accuracy: 11.2717%, Time: 14.1999s
Best Accuracy for validation : 0.1272 at epoch 088
Epoch: 95/3000
Epoch: 095, Training: Loss: 3.1307, Accuracy: 6.4547%, 
		Validation: Loss: 3.1307, Accuracy: 11.8497%, Time: 13.7520s
Best Accuracy for validation : 0.1272 at epoch 088
Epoch: 96/3000
Epoch: 096, Training: Loss: 3.1270, Accuracy: 6.2620%, 
		Validation: Loss:

Epoch: 136, Training: Loss: 2.7825, Accuracy: 7.7071%, 
		Validation: Loss: 2.7825, Accuracy: 17.6301%, Time: 13.9363s
Best Accuracy for validation : 0.1850 at epoch 132
Epoch: 137/3000
Epoch: 137, Training: Loss: 2.8535, Accuracy: 8.2852%, 
		Validation: Loss: 2.8535, Accuracy: 15.3179%, Time: 13.9973s
Best Accuracy for validation : 0.1850 at epoch 132
Epoch: 138/3000
Epoch: 138, Training: Loss: 2.8041, Accuracy: 7.9961%, 
		Validation: Loss: 2.8041, Accuracy: 12.7168%, Time: 13.9560s
Best Accuracy for validation : 0.1850 at epoch 132
Epoch: 139/3000
Epoch: 139, Training: Loss: 2.7770, Accuracy: 9.0559%, 
		Validation: Loss: 2.7770, Accuracy: 13.8728%, Time: 13.9375s
Best Accuracy for validation : 0.1850 at epoch 132
Epoch: 140/3000
Epoch: 140, Training: Loss: 2.8074, Accuracy: 7.9961%, 
		Validation: Loss: 2.8074, Accuracy: 17.3410%, Time: 13.9469s
Best Accuracy for validation : 0.1850 at epoch 132
Epoch: 141/3000
Epoch: 141, Training: Loss: 2.7585, Accuracy: 9.1522%, 
		Validation: 

Epoch: 180, Training: Loss: 2.4784, Accuracy: 12.0424%, 
		Validation: Loss: 2.4784, Accuracy: 21.6763%, Time: 13.9678s
Best Accuracy for validation : 0.2341 at epoch 168
Epoch: 181/3000
Epoch: 181, Training: Loss: 2.4494, Accuracy: 10.5973%, 
		Validation: Loss: 2.4494, Accuracy: 18.7861%, Time: 13.8639s
Best Accuracy for validation : 0.2341 at epoch 168
Epoch: 182/3000
Epoch: 182, Training: Loss: 2.4547, Accuracy: 10.3083%, 
		Validation: Loss: 2.4547, Accuracy: 22.5434%, Time: 13.8682s
Best Accuracy for validation : 0.2341 at epoch 168
Epoch: 183/3000
Epoch: 183, Training: Loss: 2.4244, Accuracy: 10.9827%, 
		Validation: Loss: 2.4244, Accuracy: 23.6994%, Time: 13.9289s
Best Accuracy for validation : 0.2370 at epoch 183
Epoch: 184/3000
Epoch: 184, Training: Loss: 2.4640, Accuracy: 10.4046%, 
		Validation: Loss: 2.4640, Accuracy: 24.5665%, Time: 13.7637s
Best Accuracy for validation : 0.2457 at epoch 184
Epoch: 185/3000
Epoch: 185, Training: Loss: 2.4766, Accuracy: 10.5973%, 
		Valida

...

Epoch: 1016, Training: Loss: 1.3120, Accuracy: 45.5684%, 
		Validation: Loss: 1.3120, Accuracy: 43.6416%, Time: 13.6429s
Best Accuracy for validation : 0.4913 at epoch 872
Epoch: 1017/3000
Epoch: 1017, Training: Loss: 1.4917, Accuracy: 43.6416%, 
		Validation: Loss: 1.4917, Accuracy: 42.1965%, Time: 13.5821s
Best Accuracy for validation : 0.4913 at epoch 872
Epoch: 1018/3000
Epoch: 1018, Training: Loss: 1.3142, Accuracy: 45.6647%, 
		Validation: Loss: 1.3142, Accuracy: 41.6185%, Time: 13.4785s
Best Accuracy for validation : 0.4913 at epoch 872
Epoch: 1019/3000
Epoch: 1019, Training: Loss: 1.4216, Accuracy: 43.7380%, 
		Validation: Loss: 1.4216, Accuracy: 40.7514%, Time: 13.5312s
Best Accuracy for validation : 0.4913 at epoch 872
Epoch: 1020/3000
Epoch: 1020, Training: Loss: 1.3331, Accuracy: 45.4721%, 
		Validation: Loss: 1.3331, Accuracy: 41.3295%, Time: 13.4922s
Best Accuracy for validation : 0.4913 at epoch 872
Epoch: 1021/3000
Epoch: 1021, Training: Loss: 1.3486, Accuracy: 46.3391%

Epoch: 1060/3000
Epoch: 1060, Training: Loss: 1.3228, Accuracy: 46.5318%, 
		Validation: Loss: 1.3228, Accuracy: 42.1965%, Time: 13.3746s
Best Accuracy for validation : 0.4913 at epoch 872
Epoch: 1061/3000
Epoch: 1061, Training: Loss: 1.2796, Accuracy: 46.0501%, 
		Validation: Loss: 1.2796, Accuracy: 44.7977%, Time: 13.7064s
Best Accuracy for validation : 0.4913 at epoch 872
Epoch: 1062/3000
Epoch: 1062, Training: Loss: 1.3881, Accuracy: 45.6647%, 
		Validation: Loss: 1.3881, Accuracy: 40.7514%, Time: 13.4186s
Best Accuracy for validation : 0.4913 at epoch 872
Epoch: 1063/3000
Epoch: 1063, Training: Loss: 1.3766, Accuracy: 43.9306%, 
		Validation: Loss: 1.3766, Accuracy: 42.7746%, Time: 13.5383s
Best Accuracy for validation : 0.4913 at epoch 872
Epoch: 1064/3000
Epoch: 1064, Training: Loss: 1.3310, Accuracy: 45.7611%, 
		Validation: Loss: 1.3310, Accuracy: 45.6647%, Time: 13.3884s
Best Accuracy for validation : 0.4913 at epoch 872
Epoch: 1065/3000
Epoch: 1065, Training: Loss: 1.4128, A

Epoch: 1104/3000
Epoch: 1104, Training: Loss: 1.4311, Accuracy: 45.9538%, 
		Validation: Loss: 1.4311, Accuracy: 43.9306%, Time: 13.8979s
Best Accuracy for validation : 0.4913 at epoch 872
Epoch: 1105/3000
Epoch: 1105, Training: Loss: 1.3659, Accuracy: 52.1195%, 
		Validation: Loss: 1.3659, Accuracy: 46.5318%, Time: 13.9518s
Best Accuracy for validation : 0.4913 at epoch 872
Epoch: 1106/3000
Epoch: 1106, Training: Loss: 1.4926, Accuracy: 44.7977%, 
		Validation: Loss: 1.4926, Accuracy: 41.9075%, Time: 13.9212s
Best Accuracy for validation : 0.4913 at epoch 872
Epoch: 1107/3000
Epoch: 1107, Training: Loss: 1.3890, Accuracy: 47.1098%, 
		Validation: Loss: 1.3890, Accuracy: 41.9075%, Time: 13.9052s
Best Accuracy for validation : 0.4913 at epoch 872
Epoch: 1108/3000
Epoch: 1108, Training: Loss: 1.4273, Accuracy: 50.2890%, 
		Validation: Loss: 1.4273, Accuracy: 42.7746%, Time: 13.8561s
Best Accuracy for validation : 0.4913 at epoch 872
Epoch: 1109/3000
Epoch: 1109, Training: Loss: 1.4844, A

Epoch: 1148, Training: Loss: 1.4181, Accuracy: 51.4451%, 
		Validation: Loss: 1.4181, Accuracy: 45.6647%, Time: 13.8512s
Best Accuracy for validation : 0.4913 at epoch 872
Epoch: 1149/3000
Epoch: 1149, Training: Loss: 1.5276, Accuracy: 49.9037%, 
		Validation: Loss: 1.5276, Accuracy: 38.7283%, Time: 13.9331s
Best Accuracy for validation : 0.4913 at epoch 872
Epoch: 1150/3000
Epoch: 1150, Training: Loss: 1.4470, Accuracy: 50.0963%, 
		Validation: Loss: 1.4470, Accuracy: 39.8844%, Time: 14.0549s
Best Accuracy for validation : 0.4913 at epoch 872
Epoch: 1151/3000
Epoch: 1151, Training: Loss: 1.4185, Accuracy: 49.6146%, 
		Validation: Loss: 1.4185, Accuracy: 42.1965%, Time: 14.0767s
Best Accuracy for validation : 0.4913 at epoch 872
Epoch: 1152/3000
Epoch: 1152, Training: Loss: 1.4323, Accuracy: 48.1696%, 
		Validation: Loss: 1.4323, Accuracy: 41.0405%, Time: 14.4059s
Best Accuracy for validation : 0.4913 at epoch 872
Epoch: 1153/3000
Epoch: 1153, Training: Loss: 1.4054, Accuracy: 48.0732%

KeyboardInterrupt: 

In [8]:
history = np.array(history)
plt.plot(history[:, 0:2])
plt.legend(['Tr Loss', 'Val Loss'])
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.ylim(0, 1)
plt.savefig(dataset+'_loss_curve.png')
plt.show()

plt.plot(history[:, 2:4])
plt.legend(['Tr Accuracy', 'Val Accuracy'])
plt.xlabel('Epoch Number')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
plt.savefig(dataset+'_accuracy_curve.png')
plt.show()

NameError: name 'history' is not defined